In [38]:
import xml.etree.ElementTree as ET
import glob
from collections import Counter
import matplotlib.pyplot as plt
import os
import cv2

## Extend datasets using Faces dataset

Retrieve all files in the validation folder to check how many files are needed to get 3000 faces worth of data to stabilise the global dataset

In [8]:
files = glob.glob('./Datasets/Faces/labels/val/*')
# files = [os.path.basename(file) for file in files]

def count_rows_in_file(file_path):
    with open(file_path, 'r') as file:
        return len(file.readlines())

In [9]:
total_rows = 0
file_count = 0

for file in files:
    rows = count_rows_in_file(file)
    total_rows += rows
    file_count += 1
    if total_rows >= 3000:
        break

print(f"It will take {file_count} files to reach {total_rows} total rows.")

It will take 998 files to reach 3000 total rows.


Save the first 998 files

In [15]:
faces_files = []

for i in range(998):
    faces_files.append(files[i])
    
for file in faces_files:
    print(file)
    

./Datasets/Faces/labels/val\0003d5dfa8f0dd41.txt
./Datasets/Faces/labels/val\0007ad5c6245a41d.txt
./Datasets/Faces/labels/val\000cc0cd61c6c8d7.txt
./Datasets/Faces/labels/val\001ea8a8ca78a3bc.txt
./Datasets/Faces/labels/val\00430ce61009c131.txt
./Datasets/Faces/labels/val\0046c490ad699ce9.txt
./Datasets/Faces/labels/val\004a688b02428848.txt
./Datasets/Faces/labels/val\005feff8c814fb4c.txt
./Datasets/Faces/labels/val\006f325ad71643a2.txt
./Datasets/Faces/labels/val\00790ccfad100384.txt
./Datasets/Faces/labels/val\007e2d0680208072.txt
./Datasets/Faces/labels/val\0081d67c4cfc9134.txt
./Datasets/Faces/labels/val\00824274fbef9c93.txt
./Datasets/Faces/labels/val\0084d707a9cca12b.txt
./Datasets/Faces/labels/val\008bf03851527e74.txt
./Datasets/Faces/labels/val\0097fb7387049d8b.txt
./Datasets/Faces/labels/val\0098312a40cabb46.txt
./Datasets/Faces/labels/val\00b242e2ccf93adc.txt
./Datasets/Faces/labels/val\00d5b7d68d52ac56.txt
./Datasets/Faces/labels/val\00e64d002478816c.txt
./Datasets/Faces/lab

In [16]:
for file in files:
    if file not in faces_files:
        os.remove(file)

Now to do the same with the respective images of the saved labels

In [32]:
image_files = glob.glob('./Datasets/Faces/images/val/*')
image_filenames = [os.path.basename(file) for file in image_files]
image_base_filenames = []
for file in image_filenames:
    image_base_filenames.append(file.split('.')[0])

to_keep_filenames = [os.path.basename(file) for file in faces_files]
to_keep_base_filenames = []
for file in to_keep_filenames:
    to_keep_base_filenames.append(file.split('.')[0])

In [34]:
# for file in to_keep_base_filenames:
#     print(file)

for file in image_base_filenames:
    if file not in to_keep_base_filenames:
        os.remove(f'./Datasets/Faces/images/val/{file}.jpg')

#### Transforming YOLO labels to Pascal VOC format

In [41]:
def yolo_to_pascal_voc(yolo_file, image_width, image_height, output_dir):
    with open(yolo_file, 'r') as f:
        lines = f.readlines()

    # Extract filename without extension
    base_filename = os.path.splitext(os.path.basename(yolo_file))[0]
    xml_file = os.path.join(output_dir, f"{base_filename}.xml")

    # Create XML structure
    annotation = ET.Element("annotation")
    folder = ET.SubElement(annotation, "folder").text = "images"
    filename = ET.SubElement(annotation, "filename").text = f"{base_filename}.jpg"

    size = ET.SubElement(annotation, "size")
    ET.SubElement(size, "width").text = str(image_width)
    ET.SubElement(size, "height").text = str(image_height)
    ET.SubElement(size, "depth").text = "3"

    ET.SubElement(annotation, "segmented").text = "0"

    for line in lines:
        class_id, x_center, y_center, width, height = map(float, line.split())

        # Convert YOLO coordinates to PASCAL VOC
        xmin = int((x_center - width / 2) * image_width)
        ymin = int((y_center - height / 2) * image_height)
        xmax = int((x_center + width / 2) * image_width)
        ymax = int((y_center + height / 2) * image_height)

        # Add object to XML
        obj = ET.SubElement(annotation, "object")
        ET.SubElement(obj, "name").text = str("without_mask")
        ET.SubElement(obj, "pose").text = "Unspecified"
        ET.SubElement(obj, "truncated").text = "0"
        ET.SubElement(obj, "difficult").text = "0"

        bndbox = ET.SubElement(obj, "bndbox")
        ET.SubElement(bndbox, "xmin").text = str(xmin)
        ET.SubElement(bndbox, "ymin").text = str(ymin)
        ET.SubElement(bndbox, "xmax").text = str(xmax)
        ET.SubElement(bndbox, "ymax").text = str(ymax)

    # Write XML to file
    tree = ET.ElementTree(annotation)
    tree.write(xml_file, encoding="utf-8", xml_declaration=True)

    print(f"Converted {yolo_file} to {xml_file}")

In [42]:
yolo_images = glob.glob('./Datasets/Faces/images/val/*')
# yolo_labels = glob.glob('./Datasets/Faces/labels/val/*')

for image in yolo_images:
    filename = os.path.basename(image).split('.')[0]
    img = cv2.imread(image)
    height, width, _ = img.shape
    yolo_label = f'./Datasets/Faces/labels/val/{filename}.txt'
    # print(f"Image: {image}")
    # print(f"Height: {height}")
    # print(f"Width: {width}")
    # print(f"Label: {yolo_label}")
    yolo_to_pascal_voc(yolo_label, width, height, './Datasets/NewPascalFormat/')

Converted ./Datasets/Faces/labels/val/0003d5dfa8f0dd41.txt to ./Datasets/NewPascalFormat/0003d5dfa8f0dd41.xml
Converted ./Datasets/Faces/labels/val/0007ad5c6245a41d.txt to ./Datasets/NewPascalFormat/0007ad5c6245a41d.xml
Converted ./Datasets/Faces/labels/val/000cc0cd61c6c8d7.txt to ./Datasets/NewPascalFormat/000cc0cd61c6c8d7.xml
Converted ./Datasets/Faces/labels/val/001ea8a8ca78a3bc.txt to ./Datasets/NewPascalFormat/001ea8a8ca78a3bc.xml
Converted ./Datasets/Faces/labels/val/00430ce61009c131.txt to ./Datasets/NewPascalFormat/00430ce61009c131.xml
Converted ./Datasets/Faces/labels/val/0046c490ad699ce9.txt to ./Datasets/NewPascalFormat/0046c490ad699ce9.xml
Converted ./Datasets/Faces/labels/val/004a688b02428848.txt to ./Datasets/NewPascalFormat/004a688b02428848.xml
Converted ./Datasets/Faces/labels/val/005feff8c814fb4c.txt to ./Datasets/NewPascalFormat/005feff8c814fb4c.xml
Converted ./Datasets/Faces/labels/val/006f325ad71643a2.txt to ./Datasets/NewPascalFormat/006f325ad71643a2.xml
Converted 

#### Transforming Pascal VOC to YOLO Labels

In [43]:
class_mapping = {
    "without_mask": 0,
    "with_mask": 1,
    "mask_weared_incorrect": 2
}

def pascal_voc_to_yolo(voc_file, output_dir):
    tree = ET.parse(voc_file)
    root = tree.getroot()

    image_width = int(root.find("size/width").text)
    image_height = int(root.find("size/height").text)

    # Create YOLO file
    base_filename = os.path.splitext(os.path.basename(voc_file))[0]
    yolo_file = os.path.join(output_dir, f"{base_filename}.txt")

    with open(yolo_file, "w") as yolo_f:
        for obj in root.findall("object"):
            class_name = obj.find("name").text

            class_id = class_mapping[class_name]

            # Get bounding box coordinates
            bndbox = obj.find("bndbox")
            xmin = int(bndbox.find("xmin").text)
            ymin = int(bndbox.find("ymin").text)
            xmax = int(bndbox.find("xmax").text)
            ymax = int(bndbox.find("ymax").text)

            # Convert to YOLO format
            x_center = ((xmin + xmax) / 2) / image_width
            y_center = ((ymin + ymax) / 2) / image_height
            width = (xmax - xmin) / image_width
            height = (ymax - ymin) / image_height

            # Write
            yolo_f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

    print(f"Converted {voc_file} to {yolo_file}")

In [ ]:
xml_files = glob.glob('./Datasets/annotations/*')

for file in xml_files:
    pascal_voc_to_yolo(file, './Datasets/NewYoloFormat/')